In [2]:
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
from langchain_openai import OpenAIEmbeddings

embedding = OpenAIEmbeddings(model='text-embedding-3-large')

In [4]:
from langchain_chroma import Chroma

# 데이터 로딩
vector_store = Chroma(
    embedding_function=embedding,
    persist_directory='./db/chromaDB2',
    collection_name='movie_rag_collection'
)

retriever = vector_store.as_retriever(search_kwargs={'k':3})

In [5]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model = "gpt-5")

In [ ]:
from typing_extensions import List, TypedDict
from langchain_core.documents import Document

class AgentState(TypedDict):
    query : str
    context : List[Document]
    answer : str

    # 세부사항
    status: Literal['search', 'recommend']
    title: Optional[str]
    year: Optional[int]
    casts: Optional[List[str]]         # 'actor: str' (X) -> 'casts: Optional[List[str]]' (O)
    director: Optional[List[str]]      # 'director: str' (X) -> 'director: Optional[List[str]]' (O)
    genre: Optional[List[str]]         # 'genre: str' (X) -> 'genre: Optional[List[str]]' (O)
    ott: Optional[List[str]]           # 'ott: str' (X) -> 'ott: Optional[List[str]]' (O)
    info: Optional[str]

In [ ]:
from enum import Enum
from pydantic import BaseModel, Field
from typing import Optional, List
from typing import Literal

# 1. 허용되는 장르 목록을 Enum으로 정의
# (메타데이터 키 'genre_SF' -> Enum 값 'SF')
class AllowedGenres(str, Enum):
    ACTION_ADVENTURE = "Action & Adventure"
    REALITY = "Reality"
    SF = "SF"
    SCI_FI_FANTASY = "Sci-Fi & Fantasy"
    가족 = "가족"
    공포 = "공포"
    다큐멘터리 = "다큐멘터리"
    드라마 = "드라마"
    로맨스 = "로맨스"
    모험 = "모험"
    미스터리 = "미스터리"
    범죄 = "범죄"
    스릴러 = "스릴러"
    애니메이션 = "애니메이션"
    액션 = "액션"
    역사 = "역사"
    음악 = "음악"
    전쟁 = "전쟁"
    코미디 = "코미디"
    판타지 = "판타지"

# 2. 허용되는 OTT 목록을 Enum으로 정의
class AllowedOTTs(str, Enum):
    DISNEY_PLUS = "Disney Plus"
    FILMBOX_PLUS = "FilmBox+"
    NETFLIX = "Netflix"
    NETFLIX_STANDARD_ADS = "Netflix Standard with Ads"
    TVING = "TVING"
    WATCHA = "Watcha"
    WAVVE = "wavve"

In [ ]:
class QueryDetails(BaseModel):
    """
    사용자의 쿼리에서 추출한 RAG 필터링용 핵심 정보
    """
    status: Literal['search', 'recommend'] = Field(description="쿼리의 내용이 정보 검색(search), 다른 영화 드라마 추천인지(recommend)")
    title: Optional[str] = Field(None, description="쿼리에서 언급된 영화나 드라마의 제목")
    year: Optional[int] = Field(None, description="쿼리에서 언급된 특정 연도")
    casts: Optional[List[str]] = Field(None, description="쿼리에서 언급된 배우 이름 목록")
    director: Optional[List[str]] = Field(None, description="쿼리에서 언급된 감독 이름 목록")
    
    # 3. List[str] 대신 List[AllowedGenres]와 List[AllowedOTTs]를 사용
    genre: Optional[List[AllowedGenres]] = Field(
        None, 
        description="쿼리에서 언급된 장르 목록. 반드시 스키마에 정의된 허용된 값 중에서만 선택해야 함."
    )
    ott: Optional[List[AllowedOTTs]] = Field(
        None, 
        description="쿼리에서 언급된 OTT 플랫폼 목록. 반드시 스키마에 정의된 허용된 값 중에서만 선택해야 함."
    )
    info: Optional[str] = Field(None, description="기타 줄거리 관련 키워드")

In [9]:
queryDetail_prompt_template = """
당신의 역할은 사용자의 쿼리를 분석하여 Pydantic 스키마 형식에 맞게 핵심 요소들을 추출하는 것입니다.

**사용자 쿼리:**
{query}

---
**추출 가이드라인:**
- 쿼리를 분석하여 Pydantic 스키마의 각 필드에 알맞은 값을 추출합니다.
- 'genre'와 'ott' 필드는 **반드시 스키마에 정의된 허용된 Enum 값 중에서만** 선택해야 합니다.
- 쿼리에 "공상과학"이 언급되면 "SF" Enum 값을 선택해야 합니다.
- 쿼리에 "넷플"이 언급되면 "Netflix" Enum 값을 선택해야 합니다.
- 쿼리에 정보가 없다면 해당 필드는 비워둡니다 (default=None).
"""

In [12]:
from langchain_core.prompts import ChatPromptTemplate

queryDetail_generate_prompt = ChatPromptTemplate.from_template(queryDetail_prompt_template)
structed_llm = llm.with_structured_output(QueryDetails)
query_analysis_chain = queryDetail_generate_prompt | structed_llm

In [ ]:
# --- 테스트 ---
# test_query = "이병헌이랑 유아인이 나오는 2020년 이후 공상과학 액션 영화 찾아줘. 넷플릭스에 있으면 좋겠어."
# response_object = query_analysis_chain.invoke({"query": test_query})

# print(response_object)

title=None year=None casts=['이병헌', '유아인'] director=None genre=[<AllowedGenres.SF: 'SF'>, <AllowedGenres.액션: '액션'>] ott=[<AllowedOTTs.NETFLIX: 'Netflix'>] info='2020년 이후'


In [ ]:
def generate_query_analysis(state: AgentState) -> AgentState:
    """
    쿼리에서 영화와 관련된 기본 요소를 분리해서 세부정보를 반환합니다.
    Args:
        state (AgentState): 기본 state
        
    Returns:
        state (AgnetState) : title, year, casts 등을 추출해서 담고있는 state
    """

    query = state['query']
    response = query_analysis_chain.invoke({"query": query})

    return response.model_dump()

In [ ]:
def route_query_type(state: AgentState) -> Literal['specific_search', 'similar_recommendation', 'broad_recommendation']:
    """
    쿼리 분석 결과를 기반으로 LangGraph의 다음 단계를 결정합니다.
    """

    query = state.get('query', "")
    status = state.get('status',"")
    title = state.get('title', "")

    if status == "search":
        return "specific_search"
    else:
        if title is not None and title != "":
            return "similar_recommendation"
        else:
            return "broad_recommendation"